In [1]:
import pandas

In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12499772417031940593, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 4945621811
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11404518400624715431
 physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("input\AllDogImages"))
# print(os.listdir("../input/images/images"))

# Any results you write to the current directory are saved as output.

['01-AllDogImages']


In [81]:
# from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.layers import MaxPooling2D

from collections import Counter
from scipy.ndimage import imread
from scipy.io import loadmat
from scipy.misc import imresize
import os
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
import numpy

In [6]:
base_model = Xception(weights='imagenet', include_top=False)

Instructions for updating:
Colocations handled automatically by placer.


In [136]:
# add a global spatial average pooling layer
x = base_model.output
#x = Conv2D(155, (2,2), input_shape=(200,200,3))(x)
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
# x = Dropout(0.5)(x)
x = Dense(624, activation = 'relu')(x)
x = Dropout(0.5)(x)
x = Dense(312, activation='relu')(x)
x = Dropout(0.5)(x)
# x = Dense(310, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(156, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(1, activation='sigmoid')(x)
# This layer being at 1024 isn't bad. ^
# x = Dropout(0.5)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(155, activation='softmax')(x)
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [137]:
model.summary()

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["acc"])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [9]:
# load images
image_dir = "input/AllDogImages/01-AllDogImages"
basenames = [f for f in os.listdir(image_dir) if(f[-3:] == "jpg")][:]
targs = ['_'.join(f.split("_")[:-1]) for f in basenames]
image_paths = [os.path.join(image_dir, f) for f in basenames]

images = [imread(imp) for imp in image_paths]
grays = {i for i, im in enumerate(images) if(len(im.shape) != 3)}
images = [im for i, im in enumerate(images) if(i not in grays)]
targs = [t for i, t in enumerate(targs) if(i not in grays)]
images = [im if(im.shape[2]==3) else im[:,:,:-1] for im in images]
# resize all images
im_size = (200,200)
# try 224x224, cause that's what VGG uses and we're going to be swapping models
images = [imresize(im, size=im_size) for im in images]

#def data_gen():
#    while(True):
        #Choose N random images from training set
        # read those images
        # preprocess, convert cats to one-hot, etc
#        yield (images, labels)

d:\anaconda\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  import sys
d:\anaconda\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  from ipykernel import kernelapp as app


In [74]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [99]:
#train_generator = train_datagen.flow_from_directory(
#        'input/AllDogImages/01-AllDogImages',  # this is the target directory
#        target_size=(200, 200),  # all images will be resized to 200x200
#        batch_size=10)
        #class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
#validation_generator = test_datagen.flow_from_directory(
#        'data/validation',
#        target_size=(150, 150),
#        batch_size=batch_size,
#        class_mode='binary')

Found 0 images belonging to 0 classes.


In [10]:
temp = Counter(targs)
#print(temp)
#print(len(images))
#print(len(targs))
print(image_dir)

input/AllDogImages/01-AllDogImages


In [11]:
def preprocess_input(x):
    y = x.astype("float16")
    y /= 255.
    y -= 0.5
    y *= 2.
    return y

images = [preprocess_input(im) for im in images]

In [125]:
# Encode labels
targ_encoder = LabelBinarizer()
labels = targ_encoder.fit_transform(targs)
# split into testing and training sets
X_train, X_test, Y_train, Y_test = train_test_split(images, labels, test_size=0.20, stratify=labels)

In [138]:
model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='bestmodel2.hdf5', 
                               verbose=1, save_best_only=True)
#model.fit([train_vgg19, train_resnet50], train_targets, 
#          validation_data=([valid_vgg19, valid_resnet50], valid_targets),
#          epochs=10, batch_size=4, callbacks=[checkpointer], verbose=1)

In [139]:
model.fit(
    x=numpy.array(X_train),
    y=Y_train,
    epochs=50,
    # Best Epoch: 500?
    batch_size=5,
    #Best Batch_size: 8
    callbacks=[checkpointer], 
    verbose=1,
    validation_data=(numpy.array(X_test), Y_test)
)

Train on 12589 samples, validate on 3148 samples
Epoch 1/50
12589/12589 [==============================] - 136s 11ms/step - loss: 4.0040 - acc: 0.1927 - val_loss: 2.8062 - val_acc: 0.4034

Epoch 00001: val_loss improved from inf to 2.80618, saving model to bestmodel2.hdf5
Epoch 2/50
12589/12589 [==============================] - 123s 10ms/step - loss: 3.6326 - acc: 0.2656 - val_loss: 2.6784 - val_acc: 0.4091

Epoch 00002: val_loss improved from 2.80618 to 2.67837, saving model to bestmodel2.hdf5
Epoch 3/50
12589/12589 [==============================] - 125s 10ms/step - loss: 3.6259 - acc: 0.2743 - val_loss: 2.7101 - val_acc: 0.4301

Epoch 00003: val_loss did not improve from 2.67837
Epoch 4/50
12589/12589 [==============================] - 119s 9ms/step - loss: 3.6052 - acc: 0.2839 - val_loss: 2.6987 - val_acc: 0.4511

Epoch 00004: val_loss did not improve from 2.67837
Epoch 5/50
12589/12589 [==============================] - 124s 10ms/step - loss: 3.6305 - acc: 0.2897 - val_loss: 2.79

KeyboardInterrupt: 

In [ ]:
# Things to mess with:
# Add Cross validation
# Add Datagenerator
# Change Test/Train split. We're at 80:20 right now. Maybe change to 70:30 or 80:20.
# Add or subtract dense layers and how many nodes per layers
# Change batch size